# Description

## Méthodologie d’analyse

Les étapes suivantes ont été réalisées pour préparer et analyser les données socio-économiques des communes :

- **Chargement et nettoyage des données** : Les données brutes issues de l’INSEE ont été importées, puis filtrées pour exclure les communes ou colonnes ne respectant pas les critères de confidentialité ou contenant trop de valeurs manquantes.

- **Enrichissement des données** : Un mapping a été effectué pour associer chaque code commune à son libellé, puis une clé unique CITY_ID a été créée pour chaque commune.

- **Sélection des variables pertinentes** : Une analyse en composantes principales (PCA) a permis d’identifier les variables ayant le plus d’incidence sur la variabilité des données. Ces variables ont été sélectionnées pour la suite de l’analyse.

- **Préparation des features** : Un sous-ensemble de variables d’intérêt a été sélectionné manuellement pour constituer la table de features utilisée dans les analyses de similarité et de clustering.

- **Normalisation et réduction de dimension** : Les données ont été normalisées (StandardScaler), puis une réduction de dimension a été appliquée (PCA) pour faciliter les analyses de proximité et de clustering.

- **Recherche de villes similaires** : Un algorithme KNN a été utilisé sur les données réduites pour identifier les communes les plus similaires à Levallois-Perret.

- **Transformation des données pour l’analyse temporelle** :Les données ont été transformées au format long, puis pivotées pour obtenir un tableau où chaque ligne correspond à une commune et une catégorie, et chaque colonne à une année (2010, 2015, 2021). Une colonne supplémentaire a été calculée pour mesurer l’évolution entre 2015 et 2021.

- **Préparation pour l’export et la visualisation** : Les données finales ont été exportées pour une utilisation dans Power BI ou d’autres outils de visualisation.

- **Sources** : 
    - https://www.insee.fr/fr/statistiques/5359146#documentation
    - https://www.insee.fr/fr/metadonnees/source/operation/s2146/documentation-methodologique


# Package

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
from sklearn.neighbors import NearestNeighbors

# Source

In [ ]:
df = pd.read_csv(r'G:\Mon Drive\Famille Vincent\Steven\CESEL\data\INSEE\dossier_complet\dossier_complet.csv', sep=';', dtype=str)
# on écarte les communes sans données pour la colonne Nombre de ménages fiscaux - s représente secret statistique, il n'y a pas assez de données pour respecter la confidentialité
df = df[~df['NBMENFISC21'].isin(['s', 'nd'])]
# on écarte l'ensemble des colonnes de textes et les colonnes n'ayant pas assez de données pour respecter la confidentialité
df = df.drop(['PIMP21','TP6021','TP60AGE121','TP60AGE221','TP60AGE321','TP60AGE421'
,'TP60AGE521','TP60AGE621','TP60TOL121','TP60TOL221','PACT21','PTSA21','PCHO21','PBEN21','PPEN21','PPAT21','PPSOC21' ,'PPFAM21' ,'PPMINI21' 
,'PPLOGT21' ,'PIMPOT21' ,'D121','D921','RD21','ETASSMAT22','ETAUTRES22'], axis=1)


In [ ]:
meta = pd.read_csv(r"G:\Mon Drive\Famille Vincent\Steven\CESEL\data\INSEE\dossier_complet\meta_dossier_complet.csv", sep=';', dtype=str)
meta_city = meta[meta['COD_VAR']=='CODGEO'][['COD_MOD', 'LIB_MOD']]

In [ ]:
df_city= df.merge(meta_city, left_on='CODGEO', right_on='COD_MOD', how='left')
df_city = df_city.reset_index(drop=True)
df_city['CITY_ID'] = df_city['CODGEO'].astype(str) + '_' + df_city['LIB_MOD'].astype(str)
df_city = df_city.drop(['CODGEO', 'COD_MOD','LIB_MOD'], axis=1)
df_city = df_city.reset_index(drop=True).fillna(0)

# Sélection automatique des champs ayant le plus d'incidence sur la variabilité

In [ ]:
X = df_city.drop(columns=['CITY_ID']) 
# Normaliser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# Appliquer PCA
pca = PCA()
pca.fit(X_scaled)

# Obtenir les chargements
loadings = pd.DataFrame(pca.components_.T, columns=[f"PC{i+1}" for i in range(len(pca.components_))], index=X.columns)

# Afficher les chargements
print(loadings)

# Sélectionner les colonnes avec des chargements élevés (par exemple, > 0.5 ou < -0.5)
important_features = []
for pc in loadings.columns:
    important_features.extend(loadings.index[abs(loadings[pc]) > 0.5].tolist())

# Supprimer les doublons
important_features = list(set(important_features))

# Afficher les colonnes importantes
print("Colonnes importantes :", important_features)

# Créer un nouveau DataFrame avec les colonnes importantes
data_important = df_city[["CITY_ID"] + important_features]  # Inclure la colonne "nom"

# Afficher le nouveau DataFrame
print(data_important.head())

del data_important['CITY_ID']
data_important = data_important.fillna(0)

# Sélection des colonnes pour les features

In [ ]:
df_feature = df_city[['CITY_ID'
    # Population emploi
    ,'C10_POP1524','C10_POP1524_CS1', 'C10_POP1524_CS2', 'C10_POP1524_CS3', 'C10_POP1524_CS4', 'C10_POP1524_CS5', 'C10_POP1524_CS6', 'C10_POP1524_CS7', 'C10_POP1524_CS8'
    ,'C15_POP1524', 'C15_POP1524_CS1', 'C15_POP1524_CS2', 'C15_POP1524_CS3', 'C15_POP1524_CS4', 'C15_POP1524_CS5', 'C15_POP1524_CS6', 'C15_POP1524_CS7', 'C15_POP1524_CS8'
    ,'C21_POP1524', 'C21_POP1524_CS1', 'C21_POP1524_CS2', 'C21_POP1524_CS3', 'C21_POP1524_CS4', 'C21_POP1524_CS5', 'C21_POP1524_CS6', 'C21_POP1524_CS7', 'C21_POP1524_CS8'
    ,'P10_ACT15P', 'P15_ACT15P' , 'P21_ACT15P'
    ,'C10_ACT1564' ,'C10_ACT1564_CS1' ,'C10_ACT1564_CS2' ,'C10_ACT1564_CS3' ,'C10_ACT1564_CS4' ,'C10_ACT1564_CS5' ,'C10_ACT1564_CS6'
    ,'C10_ACTOCC1564' ,'C10_ACTOCC1564_CS1' ,'C10_ACTOCC1564_CS2' ,'C10_ACTOCC1564_CS3' ,'C10_ACTOCC1564_CS4' ,'C10_ACTOCC1564_CS5' ,'C10_ACTOCC1564_CS6'
    ,'C15_ACT1564' ,'C15_ACT1564_CS1' ,'C15_ACT1564_CS2' ,'C15_ACT1564_CS3' ,'C15_ACT1564_CS4' ,'C15_ACT1564_CS5' ,'C15_ACT1564_CS6'
    ,'C15_ACTOCC1564' ,'C15_ACTOCC1564_CS1' ,'C15_ACTOCC1564_CS2' ,'C15_ACTOCC1564_CS3' ,'C15_ACTOCC1564_CS4' ,'C15_ACTOCC1564_CS5' ,'C15_ACTOCC1564_CS6'
    ,'C21_ACT1564' ,'C21_ACT1564_CS1' ,'C21_ACT1564_CS2' ,'C21_ACT1564_CS3' ,'C21_ACT1564_CS4' ,'C21_ACT1564_CS5' ,'C21_ACT1564_CS6'
    ,'C21_ACTOCC1564','C21_ACTOCC1564_CS1','C21_ACTOCC1564_CS2','C21_ACTOCC1564_CS3','C21_ACTOCC1564_CS4','C21_ACTOCC1564_CS5','C21_ACTOCC1564_CS6'
    ,'P21_POP15P' ,'P21_POP1519' ,'P21_POP2024' ,'P21_POP2539' ,'P21_POP4054' ,'P21_POP5564' ,'P21_POP6579' ,'P21_POP80P'
    ,'P15_POP15P' ,'P15_POP1519' ,'P15_POP2024' ,'P15_POP2539' ,'P15_POP4054' ,'P15_POP5564' ,'P15_POP6579','P15_POP80P'
    ,'P10_POP15P','P10_POP1519','P10_POP2024','P10_POP2539','P10_POP4054','P10_POP5564','P10_POP6579','P10_POP80P'
    # Chomage
    ,'P21_CHOM1564','P21_CHOM1524','P21_CHOM2554','P21_CHOM5564','P21_CHOM_DIPLMIN','P21_CHOM_BEPC','P21_CHOM_CAPBEP','P21_CHOM_BAC','P21_CHOM_SUP2','P21_CHOM_SUP34','P21_CHOM_SUP5'
    ,'P15_CHOM1564'
    ,'P10_CHOM1564'
    ,'P10_ACT1524', 'P10_ACTOCC1524', 'P10_F1524', 'P10_FACT1524', 'P10_FACTOCC1524', 'P10_FSAL1524', 'P10_FSAL1524_TP', 'P10_H1524', 'P10_HACT1524', 'P10_HACTOCC1524', 'P10_HCHOM1524', 'P10_HSAL1524', 'P10_HSAL1524_TP'
    ,'P15_ACT1524', 'P15_ACTOCC1524', 'P15_F1524', 'P15_FACT1524', 'P15_FACTOCC1524', 'P15_FSAL1524', 'P15_FSAL1524_TP', 'P15_H1524', 'P15_HACT1524', 'P15_HACTOCC1524', 'P15_HCHOM1524', 'P15_HSAL1524', 'P15_HSAL1524_TP', 'P15_POP1524', 'P15_POP1524_IRAN2', 'P15_POP1524_IRAN2P', 'P15_POP1524_IRAN3P'
    ,'P21_ACT1524', 'P21_ACTOCC1524', 'P21_F1524', 'P21_FACT1524', 'P21_FACTOCC1524', 'P21_FSAL1524', 'P21_FSAL1524_TP', 'P21_H1524', 'P21_HACT1524', 'P21_HACTOCC1524', 'P21_HCHOM1524', 'P21_HSAL1524', 'P21_HSAL1524_TP', 'P21_POP1524', 'P21_POP1524_IRAN2', 'P21_POP1524_IRAN2P', 'P21_POP1524_IRAN3P'
    # Population totale
    ,'P21_POP', 'P15_POP', 'P10_POP'
    ,'P15_FSAL1564', 'P10_FSAL1564', 'P21_FSAL1564'
    ,'P15_RPAPPART', 'P21_RPAPPART', 'P10_RPAPPART'
    ,'C21_MEN' ,'C21_MENPSEUL' ,'C21_MENHSEUL' ,'C21_MENFSEUL' ,'C21_MENSFAM' ,'C21_MENFAM' ,'C21_MENCOUPSENF' ,'C21_MENCOUPAENF' ,'C21_MENFAMMONO'
    ,'C21_PMEN' ,'C21_PMEN_MENPSEUL' ,'C21_PMEN_MENHSEUL' ,'C21_PMEN_MENFSEUL' ,'C21_PMEN_MENSFAM' ,'C21_PMEN_MENFAM' ,'C21_PMEN_MENCOUPSENF' ,'C21_PMEN_MENCOUPAENF' ,'C21_PMEN_MENFAMMONO'
    ,'C15_MEN','C15_MENPSEUL','C15_MENHSEUL','C15_MENFSEUL','C15_MENSFAM','C15_MENFAM','C15_MENCOUPSENF','C15_MENCOUPAENF','C15_MENFAMMONO'
    ,'C15_PMEN','C15_PMEN_MENPSEUL','C15_PMEN_MENHSEUL','C15_PMEN_MENFSEUL','C15_PMEN_MENSFAM','C15_PMEN_MENFAM','C15_PMEN_MENCOUPSENF','C15_PMEN_MENCOUPAENF','C15_PMEN_MENFAMMONO'
    ,'C10_MEN' ,'C10_MENPSEUL' ,'C10_MENHSEUL' ,'C10_MENFSEUL' ,'C10_MENSFAM' ,'C10_MENFAM' ,'C10_MENCOUPSENF' ,'C10_MENCOUPAENF' ,'C10_MENFAMMONO'
    ,'C10_PMEN' ,'C10_PMEN_MENPSEUL' ,'C10_PMEN_MENHSEUL' ,'C10_PMEN_MENFSEUL' ,'C10_PMEN_MENSFAM' ,'C10_PMEN_MENFAM' ,'C10_PMEN_MENCOUPSENF' ,'C10_PMEN_MENCOUPAENF' ,'C10_PMEN_MENFAMMONO'
    # Nombre d'etablissement
    ,'ETTOT22' ,'ETAZ22' ,'ETBE22' ,'ETFZ22' ,'ETGU22' ,'ETGZ22' ,'ETOQ22'
    ,'P21_ACTOCC', 'P10_ACTOCC', 'P15_ACTOCC' 
    ,'P21_POP01P','P21_POP01P_IRAN1','P21_POP01P_IRAN2','P21_POP01P_IRAN3','P21_POP01P_IRAN4','P21_POP01P_IRAN5','P21_POP01P_IRAN6','P21_POP01P_IRAN7','P21_POP0114_IRAN2P','P21_POP0114_IRAN2','P21_POP0114_IRAN3P'
    ,'P15_POP01P' ,'P15_POP01P_IRAN1' ,'P15_POP01P_IRAN2' ,'P15_POP01P_IRAN3' ,'P15_POP01P_IRAN4' ,'P15_POP01P_IRAN5' ,'P15_POP01P_IRAN6' ,'P15_POP01P_IRAN7' ,'P15_POP0114_IRAN2P' ,'P15_POP0114_IRAN2' ,'P15_POP0114_IRAN3P'
    # Diplome & Scolarisation
    ,'P21_SCOL1517','P21_SCOL1824','P21_SCOL2529','P21_SCOL30P'
    ,'P15_SCOL1517' ,'P15_SCOL1824' ,'P15_SCOL2529' ,'P15_SCOL30P'
    ,'P10_SCOL1517' ,'P10_SCOL1824' ,'P10_SCOL2529' ,'P10_SCOL30P'
    ,'P21_FSCOL1517','P21_FSCOL1824','P21_FSCOL2529','P21_FSCOL30P'
    ,'P15_FSCOL1517','P15_FSCOL1824','P15_FSCOL2529','P15_FSCOL30P'
    ,'P10_FSCOL1517','P10_FSCOL1824','P10_FSCOL2529','P10_FSCOL30P'
    ,'P21_HSCOL1517','P21_HSCOL1824','P21_HSCOL2529','P21_HSCOL30P'
    ,'P15_HSCOL1517','P15_HSCOL1824','P15_HSCOL2529','P15_HSCOL30P'
    ,'P10_HSCOL1517','P10_HSCOL1824','P10_HSCOL2529','P10_HSCOL30P'
    ,'P21_NSCOL15P' ,'P21_NSCOL15P_DIPLMIN' ,'P21_NSCOL15P_BEPC' ,'P21_NSCOL15P_CAPBEP' ,'P21_NSCOL15P_BAC' ,'P21_NSCOL15P_SUP2' ,'P21_NSCOL15P_SUP34' ,'P21_NSCOL15P_SUP5' 
    ,'P15_NSCOL15P'
    ,'P10_NSCOL15P'
    ,'P21_HNSCOL15P' ,'P21_HNSCOL15P_DIPLMIN' ,'P21_HNSCOL15P_BEPC' ,'P21_HNSCOL15P_CAPBEP' ,'P21_HNSCOL15P_BAC' ,'P21_HNSCOL15P_SUP2' ,'P21_HNSCOL15P_SUP34' ,'P21_HNSCOL15P_SUP5' 
    ,'P15_HNSCOL15P' 
    ,'P10_HNSCOL15P' 
    ,'P21_FNSCOL15P' ,'P21_FNSCOL15P_DIPLMIN' ,'P21_FNSCOL15P_BEPC' ,'P21_FNSCOL15P_CAPBEP' ,'P21_FNSCOL15P_BAC' ,'P21_FNSCOL15P_SUP2' ,'P21_FNSCOL15P_SUP34' ,'P21_FNSCOL15P_SUP5' 
    ,'P15_FNSCOL15P' 
    ,'P10_FNSCOL15P' 

    ]]

# Utilisation KNN

In [ ]:
X = df_feature.drop(columns=['CITY_ID'])  # Suppression de la colonne 'nom' pour le clustering
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2. Réduction de la dimensionnalité
pca = PCA(n_components=317)  # Ajustez le nombre de composantes
X_pca = pca.fit_transform(X_scaled)

# 3. Application de KNN
knn = NearestNeighbors(n_neighbors=26, metric='euclidean')  # Ajustez n_neighbors et metric
knn.fit(X_pca)

# Trouver l'indice de Levallois
levallois_index = df_city[df_city['CITY_ID'] == '92044_Levallois-Perret'].index[0]
# Trouver les villes similaires
distances, indices = knn.kneighbors([X_pca[levallois_index]])

# 4. Interprétation des résultats
print("Villes similaires à Levallois-Perret :")
similar_city = []
for i in range(len(indices[0])):
    print(df_city.iloc[indices[0][i]]['CITY_ID'])
    similar_city.append(df_city.iloc[indices[0][i]]['CITY_ID'])

In [ ]:
df_select_city = df_city.loc[((df_city['CITY_ID'].isin(similar_city)) | (df_city['CITY_ID'].isin(['92009_Bois-Colombes', '92026_Courbevoie', '92033_Garches', '92035_La Garenne-Colombes', '92051_Neuilly-sur-Seine', '92062_Puteaux', '92063_Rueil-Malmaison', '92073_Suresnes'])))]

In [ ]:
df_analyse = df_select_city #df_city[['CITY_ID','P10_HCHOM1524', 'P15_HCHOM1524', 'P21_HCHOM1524', 'P10_FCHOM1524', 'P15_FCHOM1524', 'P21_FCHOM1524']]
# Map the prefixes to years
year_mapping = {
    "P10": 2010,
    "P15": 2015,
    "P21": 2021
}

# Melt the DataFrame to long format
df_long = df_analyse.melt(id_vars=["CITY_ID"], var_name="column", value_name="value")

# Extract the year and category from the column names
df_long["YEAR"] = df_long["column"].str.extract(r'(^P\d{2})').replace(year_mapping)
df_long["CATEGORY"] = df_long["column"].str.extract(r'_(.*)')

# Drop the original column name
df_long = df_long.drop(columns=["column"])
df_long['value'] = df_long['value'].astype(float)
# Pivot the DataFrame to get the desired structure, including CITY_ID
df_transformed = df_long.pivot_table(
    index=["CITY_ID", "CATEGORY"],  # Keep CITY_ID and CATEGORY as indices
    columns="YEAR",                # Use YEAR as columns
    values="value",                # Use the value column
    aggfunc="sum"                  # Aggregate in case of duplicates
).reset_index()

# Display the transformed DataFrame
display(df_transformed)

In [ ]:
df_transformed['EVOL_15_21']=(df_transformed[2021]-df_transformed[2015])/df_transformed[2015]*100

In [ ]:
df_transformed[df_transformed['CITY_ID'].isin(similar_city)].sort_values(by='EVOL_15_21', ascending=True)

# Power BI data prep

In [ ]:
df_long = df_long.drop_duplicates()
pbi_prep = pd.pivot_table(df_long, values='value', index=['CITY_ID', 'YEAR'],columns=['CATEGORY'], aggfunc="sum").reset_index()*
pbi_prep.to_csv(r'G:\Mon Drive\Famille Vincent\Steven\CESEL\data\pbi_prep.csv', sep=';', index=False)

# Calcul correlation

In [ ]:
for each in df_feature.columns[1:]:
    df_feature[each] = df_feature[each].astype(float)

In [ ]:
df_pt = df_feature.pivot_table(columns='CITY_ID')
leval_rating = df_pt['92044_Levallois-Perret']
city_like_levallois = df_pt.corrwith(leval_rating)
t = pd.DataFrame(city_like_levallois, columns=['Correlation']).reset_index()
t.sort_values(by='Correlation', ascending=False)

# Archive

In [ ]:
#population par age et sexe levallois
df_city[df_city['CODGEO']=='92044'][['CODGEO','P21_POP1524','P21_H1524', 'P21_F1524' , 'P15_POP1524', 'P15_H1524', 'P15_F1524' , 'P10_H1524', 'P10_F1524', 'P21_POP', 'P15_POP', 'P10_POP']]
df_city[df_city['CODGEO']==92044][['CODGEO','P21_HCHOM1524', 'P15_HCHOM1524', 'P10_HCHOM1524','P21_FCHOM1524', 'P15_FCHOM1524', 'P10_FCHOM1524']]

# Vérification du nombre de cluster à utiliser

In [ ]:
# Suppression de la colonne 'nom' pour le clustering
X = df_feature.drop(columns=['CITY_ID'])
# Normalisation des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

for methode in ['elbow', 'silhouette', 'calinski_harabasz']:
    methode_list = []
    for i in range(2, 21):
        kmeans = KMeans(n_clusters=i, random_state=42)
        # Méthode du coude
        if methode == 'elbow':
            kmeans.fit(X_scaled)
            methode_list.append(kmeans.inertia_)
        # Méthode de la silhouette.
        elif methode == 'silhouette':
            labels = kmeans.fit_predict(X_scaled)
            methode_list.append(silhouette_score(X_scaled, labels))
        # Calcul de l'indice Calinski-Harabasz pour différentes valeurs de K
        elif methode == 'calinski_harabasz':
            labels = kmeans.fit_predict(X_scaled)
            methode_list.append(calinski_harabasz_score(X_scaled, labels))
    plt.plot(range(2, 21), methode_list)
    plt.title('Méthode {mtd}'.format(mtd=methode))
    plt.xlabel('Nombre de clusters')
    plt.ylabel(str(methode))
    plt.show()
    # Détermination du nombre optimal de clusters
    optimal_k = range(2, 21)[methode_list.index(max(methode_list))]
    print(f"Le nombre optimal de clusters est : {optimal_k}")

## Calcul de similarité via le modèle KNN

Utilisation de la méthode K-means pour le clustering

In [ ]:

# Normalisation des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_feature.drop(columns=['CITY_ID']))

# Application du K-means
kmeans = KMeans(n_clusters=3, random_state=42)  # Choisissez le nombre de clusters
df_city["cluster"] = kmeans.fit_predict(X_scaled)

# Résultats
levallois_cluster = df_city[df_city["LIB_MOD"] == 'Levallois-Perret']["cluster"].values[0]
villes_similaires = df_city[df_city["cluster"] == levallois_cluster]

print(villes_similaires[["LIB_MOD", "cluster"]])

In [ ]:
df_23 = pd.read_parquet(r'G:\Mon Drive\Famille Vincent\Steven\CESEL\data\INSEE\FD_EEC23.parquet', engine='pyarrow')

In [ ]:
# Corrected DataFrames with all arrays of the same length

df_aac = pd.DataFrame({
    "AAC": ["1", "2", "9"],
    "Libellé": ["Oui", "Non", "Non réponse"]
})

df_acl_emploi = pd.DataFrame({
    "ACL_EMPLOI": ["I1", "I2", "I3", "I4", "A1", "A2", "A3", "A4", "B1", "B2", "B3", "B4", "C1", "C2", "C3", "C4", "D1", "D2", "D3", "D4", "99"],
    "Libellé": ["Emploi indépendant de niveau supérieur", "Emploi indépendant de niveau intermédiaire", "Emploi de petit indépendant, avec salarié ou aide familiale", "Emploi de petit indépendant, sans salarié ou aide familiale", "Emploi salarié de niveau supérieur d'orientation technique, en CDI", "Emploi salarié de niveau supérieur d'orientation tertiaire, en CDI", "Emploi salarié de niveau supérieur, fonctionnaire", "Emploi salarié de niveau supérieur, en contrat à durée limitée", "Emploi salarié de niveau d'orientation technique, en CDI", "Emploi salarié de niveau intermédiaire d'orientation tertiaire, en CDI", "Emploi salarié de niveau intermédiaire, fonctionnaire", "Emploi salarié de niveau intermédiaire, en contrat à durée limitée", "Emploi salarié qualifié d'orientation ouvrière, en CDI", "Emploi salarié qualifié d'orientation employée, en CDI", "Emploi salarié qualifié, fonctionnaire", "Emploi salarié qualifié, en contrat à durée limitée", "Emploi salarié peu qualifié d'orientation ouvrière, en CDI", "Emploi salarié peu qualifié, en CDI", "Emploi salarié peu qualifié, fonctionnaire", "Emploi salarié peu qualifié, en contrat à durée limitée", "Non connue"]
})

df_acteu = pd.DataFrame({
    "ACTEU": ["1", "2", "3", "4"],
    "Libellé": ["Emploi", "Chômage", "Inactivité", "Non renseigné"]
})

df_age6 = pd.DataFrame({
    "AGE6": ["00", "15", "25", "50", "65", "90"],
    "Libellé": ["0 à 14 ans", "15-24 ans", "25-49 ans", "50-64 ans", "65-89 ans", "90 ans ou plus"]
})


df_aisco2 = pd.DataFrame({
    "AISCO2": [f"{i:02d}" for i in range(1, 100)] + ["99"],
    "Libellé": ["Directeurs, cadres de direction et gérants", "Professions intellectuelles et scientifiques", "Techniciens et ingénieurs", "Cadres et professions intermédiaires de la fonction publique", "Professions intermédiaires administratives et commerciales des entreprises", "Professions intermédiaires de l'enseignement, de la formation et du sport", "Professions intermédiaires de la santé et du travail social", "Professions intermédiaires de la fonction publique (administration, sécurité)", "Employés de bureau et de commerce", "Employés des services directs aux particuliers", "Artisans, commerçants et chefs d'entreprise", "Ouvriers qualifiés", "Ouvriers non qualifiés", "Travailleurs agricoles, forestiers et pêcheurs", "Soldats, marins, aviateurs", "Autres professions"] + [""] * (100 - 16)
})



df_ancchom = pd.DataFrame({
    "ANCCHOM": ["1", "2", "3", "4", "5", "6", "7", "8", "9"],
    "Libellé": ["Moins d'un mois", "De 1 mois à moins de 3 mois", "De 3 mois à moins de 6 mois", "De 6 mois à moins de 12 mois", "De 12 mois à moins de 18 mois", "De 18 mois à moins de 24 mois", "De 2 ans à moins de 4 ans", "4 ans ou plus", "Non réponse"]
})

df_ancempl4 = pd.DataFrame({
    "ANCEMPL4": ["1", "2", "3", "4", "9"],
    "Libellé": ["Moins d'un an", "De 1 an à moins de 5 ans", "De 5 ans à moins de 10 ans", "10 ans ou plus", "Non réponse"]
})

df_ancssemp = pd.DataFrame({
    "ANCSSEMP": ["1", "2", "3", "4", "5", "6", "7", "8", "9", "99"],
    "Libellé": ["Moins d'un mois", "De 1 mois à moins de 3 mois", "De 3 mois à moins de 6 mois", "De 6 mois à moins de 12 mois", "De 12 mois à moins de 18 mois", "De 18 mois à moins de 24 mois", "De 2 ans à moins de 4 ans", "De 4 à moins de 10 ans", "10 ans ou plus", "Non réponse"]
})

df_annee = pd.DataFrame({
    "ANNEE": ["2024"],
    "Libellé": ["Année de référence"]
})

df_apcs1 = pd.DataFrame({
    "APCS1": ["0", "1", "2", "3", "4", "5", "6"],
    "Libellé": ["Non codée", "Agriculteurs exploitants", "Artisans, commerçants et chefs d'entreprise", "Cadres et professions intellectuelles supérieures", "Professions intermédiaires", "Employés", "Ouvriers"]
})

df_apcs2 = pd.DataFrame({
    "APCS2": ["00", "10", "21", "22", "23", "6", "31", "33", "34", "35", "37", "38", "42", "43", "44", "45", "46", "47", "48", "52", "53", "54", "55", "56", "62", "63", "64", "65", "67", "68", "69"],
    "Libellé": ["Non codée", "Exploitants agricoles, forestiers, pêcheurs et aquaculteurs", "Artisans", "Commerçants et assimilés", "Chefs d'entreprise de plus de 10 personnes", "Autres", "Professions libérales", "Cadres de la fonction publique", "Professeurs et professions scientifiques supérieures", "Professions de l'information, de l'art et des spectacles", "Cadres des services administratifs et commerciaux d'entreprise", "Ingénieurs et cadres techniques d'entreprises", "Professions de l'enseignement primaire et professionnel, de la formation continue et du sport", "Professions intermédiaires de la santé et du travail social", "Ministres du culte et religieux consacrés", "Professions intermédiaires de la fonction publique (administration, sécurité)", "Professions intermédiaires administratives et commerciales des entreprises", "Techniciens", "Agents de maîtrise (hors maîtrise administrative)", "Employés administratifs de la fonction publique, agents de service et auxiliaires de santé", "Policiers, militaires, pompiers, agents de sécurité privés", "Employés administratifs d'entreprise", "Employés de commerce", "Personnels des services directs aux particuliers", "Ouvriers qualifiés de type industriel", "Ouvriers qualifiés de type artisanal", "Conducteurs de véhicules de transport, chauffeurs-livreurs, coursiers", "Conducteurs d'engins, caristes, magasiniers et ouvriers du transport (non routier)", "Ouvriers peu qualifiés de type industriel", "Ouvriers peu qualifiés de type artisanal", "Ouvriers agricoles, des travaux forestiers, de la pêche et de l'aquaculture"]
})

df_champ_m_15_89 = pd.DataFrame({
    "CHAMP_M_15_89": ["0", "1"],
    "Libellé": ["Non", "Oui"]
})

df_chpub = pd.DataFrame({
    "CHPUB": ["1", "2", "3", "4", "5", "6", "7", "9"],
    "Libellé": ["Une entreprise privée ou une association", "Une entreprise publique (EDF, La Poste, SNCF...)", "L'État ou un établissement public administratif (Universités, Pôle emploi, CNRS...)", "Une collectivité territoriale", "Un hôpital ou un établissement médico-social (maison de retraite, Ehpad...) public", "Un organisme de Sécurité sociale (Caf, MSA, Urssaf, CPAM, Carsat...)", "Un particulier", "Non réponse"]
})

df_cl_emploi = pd.DataFrame({
    "CL_EMPLOI": ["I1", "I2", "I3", "I4", "A1", "A2", "A3", "A4", "B1", "B2", "B3", "B4", "C1", "C2", "C3", "C4", "D1", "D2", "D3", "D4", "99"],
    "Libellé": ["Emploi indépendant de niveau supérieur", "Emploi indépendant de niveau intermédiaire", "Emploi de petit indépendant, avec salarié ou aide familiale", "Emploi de petit indépendant, sans salarié ou aide familiale", "Emploi salarié de niveau supérieur d'orientation technique, en CDI", "Emploi salarié de niveau supérieur d'orientation tertiaire, en CDI", "Emploi salarié de niveau supérieur, fonctionnaire", "Emploi salarié de niveau supérieur, en contrat à durée limitée", "Emploi salarié de niveau d'orientation technique, en CDI", "Emploi salarié de niveau intermédiaire d'orientation tertiaire, en CDI", "Emploi salarié de niveau intermédiaire, fonctionnaire", "Emploi salarié de niveau intermédiaire, en contrat à durée limitée", "Emploi salarié qualifié d'orientation ouvrière, en CDI", "Emploi salarié qualifié d'orientation employée, en CDI", "Emploi salarié qualifié, fonctionnaire", "Emploi salarié qualifié, en contrat à durée limitée", "Emploi salarié peu qualifié d'orientation ouvrière, en CDI", "Emploi salarié peu qualifié, en CDI", "Emploi salarié peu qualifié, fonctionnaire", "Emploi salarié peu qualifié, en contrat à durée limitée", "Non connue"]
})

df_coupl_log = pd.DataFrame({
    "COUPL_LOG": ["1", "2"],
    "Libellé": ["Oui", "Non"]
})

df_demne = pd.DataFrame({
    "DEMNE": ["1", "2", "9"],
    "Libellé": ["Oui", "Non", "Non réponse"]
})

df_dip7 = pd.DataFrame({
    "DIP7": ["1", "2", "3", "4", "5", "6", "7", "9"],
    "Libellé": ["Diplôme de niveau bac+5 ou plus (doctorat, master ou équivalent)", "Diplôme de niveau bac+3/4 (licence, maîtrise ou équivalent)", "Diplôme de niveau bac+2 (DUT, BTS, Deug ou équivalent)", "Bac général, technologique, professionnel ou équivalent", "CAP, BEP ou équivalent", "BEPC, DNB, brevet des collèges", "Aucun diplôme, certificat d'études primaires", "Non réponse"]
})

df_dispone = pd.DataFrame({
    "DISPONE": ["1", "2", "9"],
    "Libellé": ["Oui", "Non", "Non réponse"]
})

df_dispplc = pd.DataFrame({
    "DISPPLC": ["1", "2", "9"],
    "Libellé": ["Oui", "Non", "Non réponse"]
})

df_enfred = pd.DataFrame({
    "ENFRED": ["1", "2"],
    "Libellé": ["Oui", "Non"]
})

df_eseg_1 = pd.DataFrame({
    "ESEG_1": ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"],
    "Libellé": ["Non codés", "Cadres dirigeants", "Professions intellectuelles et scientifiques", "Professions intermédiaires salariées", "Petits entrepreneurs (non-salariés)", "Employés qualifiés", "Ouvriers qualifiés salariés", "Professions salariées peu qualifiées", "Retraités", "Autres personnes hors du marché du travail"]
})

#df_eseg_2 = pd.DataFrame({
#    "Colonne": ["ESEG_2"] * 100,
#    "Code": [f"{i:02d}" for i in range(1, 100)],
#    "Libellé": ["Non codés", "Cadres dirigeants", "Professions intellectuelles et scientifiques", "Techniciens et ingénieurs", "Cadres et professions intermédiaires de la fonction publique", "Professions intermédiaires administratives et commerciales des entreprises", "Professions intermédiaires de l'enseignement, de la formation et du sport", "Professions intermédiaires de la santé et du travail social", "Professions intermédiaires de la fonction publique (administration, sécurité)", "Employés de bureau et de commerce", "Employés des services directs aux particuliers", "Artisans, commerçants et chefs d'entreprise", "Ouvriers qualifiés", "Ouvriers non qualifiés", "Travailleurs agricoles, forestiers et pêcheurs", "Soldats, marins, aviateurs", "Autres professions"] + [""] * (100 - 16)
#})


In [ ]:
dfs = [df_aac, df_acl_emploi, df_acteu, df_age6, df_aisco2, df_ancchom, df_ancempl4, df_ancssemp, df_annee, df_apcs1, df_apcs2, df_champ_m_15_89, df_chpub, df_cl_emploi, df_coupl_log, df_demne, df_dip7, df_dispone, df_dispplc, df_enfred, df_eseg_1]

# Create a list of DataFrame names
df_names = ["df_aac", "df_acl_emploi", "df_acteu", "df_age6", "df_aisco2", "df_ancchom", "df_ancempl4", "df_ancssemp", "df_annee", "df_apcs1", "df_apcs2", "df_champ_m_15_89", "df_chpub", "df_cl_emploi", "df_coupl_log", "df_demne", "df_dip7", "df_dispone", "df_dispplc", "df_enfred", "df_eseg_1"]

# Encapsulate all DataFrames into a single DataFrame
master_df_encapsulated = pd.concat([df.assign(dataframe_name=name) for df, name in zip(dfs, df_names)], ignore_index=True)


In [ ]:
full_df = pd.DataFrame()
for each in df_23.columns:
    temp = df_23[[each]].merge(master_df_encapsulated[master_df_encapsulated[each].notnull()][['Libellé', each]], on = str(each), how='outer')
    full_df = pd.concat([full_df, temp], axis=1)